In [1]:
import numpy as np

from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem

import sys
sys.path.append('../../')
from entanglement_forging import (EntanglementForgedConfig,
                                  EntanglementForgedGroundStateSolver)
...

# specify problem
molecule = Molecule(geometry=[('H', [0., 0., 0.]),
                              ('H', [0., 0., 0.735])],
                     charge=0, multiplicity=1)
driver = PySCFDriver.from_molecule(molecule = molecule, basis='sto3g')
problem = ElectronicStructureProblem(driver)
problem.second_q_ops()

# specify parameters
bitstrings = [[1, 0], [0, 1]]
ansatz = TwoLocal(2, [], 'cry', [[0, 1], [1, 0]], reps=1)

# specify configuration for forgnig
config = EntanglementForgedConfig(backend=Aer.get_backend('statevector_simulator'),
                                  maxiter=1,
                                  initial_params=[0, 0.5 * np.pi])
# specify converter
converter = QubitConverter(JordanWignerMapper())

# create solver
forged_ground_state_solver = EntanglementForgedGroundStateSolver(
  converter, ansatz, bitstrings, config)

# run solver
forged_result = forged_ground_state_solver.solve(problem)

# get ground state energy
print(f"Ground state energy: {forged_result.ground_state_energy}.")

Ground state energy: -1.1203690221324343.
